# Assignment 2 - RT2

Libraries:

In [1]:
import jupyros as jr
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023
import assignment_2_2023.msg

import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import RobotMsg
from matplotlib.animation import FuncAnimation

%matplotlib widget

In [2]:
goals_cancelled = 0
goals_reached = 0 
cancelled = False 
goal = None

Odometry callback function:

In [3]:
def callback_odom(msg):
    global pub

    my_custom_msg = RobotMsg()
    
    my_custom_msg.x = msg.pose.pose.position.x
    my_custom_msg.y = msg.pose.pose.position.y
    my_custom_msg.vel_x = msg.twist.twist.linear.x
    my_custom_msg.vel_y = msg.twist.twist.linear.y
    
    if not rospy.is_shutdown():
        pub.publish(my_custom_msg)

Send the robot to the goal function:

In [4]:
def callback_send(msg):
    global cancelled, goal
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    cancelled = False
    client.send_goal(goal)

Cancel function:

In [5]:
def callback_cancel(msg):
    global cancelled, goal 
    goal = None
    cancelled = True
    client.cancel_goal()

Define Buttons for Selecting the mode:

In [6]:
send_goal = widgets.Button(value = False, description = "Set the Target",
                                  disabled = False, button_style = '')

cancel_goal = widgets.Button(value = False, description = "Cancel the Target", 
                                    disabled = False, button_style = '')

In [7]:
send_goal.on_click(callback_send)
cancel_goal.on_click(callback_cancel)

Define widgets as input to set the target:

In [8]:
x = widgets.BoundedFloatText(value = 0.0, description = 'x', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))
y = widgets.BoundedFloatText(value = 0.0, description = 'y', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

The interface for the position of the robot:

In [9]:
class Odom_Interface:
    def __init__(self):
        self.fig_odom, self.ax = plt.subplots()
        # Robot Position Plot
        self.ln, = plt.plot([], [], 'ro', label = 'Robot Position')
        # Target's Position Plot
        self.goal_ln, = plt.plot([], [], 'b*', markersize = 10, label = 'Robot Goal Position')
        # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Position", fontsize = 15, fontweight = 'bold')
        # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize = 10, fontweight = "bold")
        self.ax.set_ylabel("Y [m]", fontsize = 10, fontweight = "bold")
        # Set Plot Axis Limits
        self.ax.set_xlim(-15, 15)
        self.ax.set_ylim(-15, 15)
        # Set Grid to True
        self.ax.grid(True)
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        # Callback Function used to update data
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)         
        
    def update_plot(self, frame):
        # Update Robot's Position on plot
        self.ln.set_data(self.x_data, self.y_data)  
        
        if cancelled: 
            self.goal_ln.set_data([], [])
        elif goal is not None:
            self.goal_ln.set_data(goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
        
        return self.ln, self.goal_ln

The interface to count the number of the targets that have been set and cancelled:

In [10]:
class Goal_Interface:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        # Set Plot Title
        self.ax.set_title("Reached & Cancelled Goals", fontsize = 15, fontweight = "bold")
        
        self.ax.grid(axis = 'y', color = 'grey', linestyle = '-', alpha = 0.3)
        # Set Plot Labels on x-axis
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 20])
        self.ax.set_yticks(np.arange(0, 21, 1))
        self.bar_colors = ['green', 'red']
        self.bar_plot = self.ax.bar(self.x_pos, [self.reached, self.cancelled], align = 'center', color = self.bar_colors, width = 0.2)

    def goal_callback(self, msg):
        # Get the number of reached/cancelled goals
        global goals_cancelled, goals_reached
        if msg.status.status == 3:
            self.reached += 1
        elif msg.status.status == 2:
            self.cancelled += 1 

    def update_plot(self, frame):
        self.green_val = np.random.randint(0,100)
        self.red_val = np.random.randint(0,100)
        
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
            bar.set_color(self.bar_colors[i])
        return self.bar_plot

Initialization:

In [11]:
rospy.init_node('jupyter_notebook')
pub = rospy.Publisher("/pos_vel", RobotMsg, queue_size = 1)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
client.wait_for_server()

True

Using widgets:

In [12]:
display(widgets.HBox([x, y]))
display(widgets.HBox([send_goal, cancel_goal]))

Plot the number of the targets that have been set and cancelled:

In [13]:
goals_plot = Goal_Interface()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2023.msg.PlanningActionResult, goals_plot.goal_callback)

animation_goal = FuncAnimation(goals_plot.fig, goals_plot.update_plot, interval = 1000)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot the robot position:

In [14]:
position_plot = Odom_Interface()
sub = rospy.Subscriber('/odom', Odometry, position_plot.odom_callback)

animation_pos = FuncAnimation(position_plot.fig_odom, position_plot.update_plot, init_func = position_plot.plot_init)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …